In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import salem
import cleo
import geopandas as gpd
import shapely.geometry as shpg
import matplotlib.pyplot as plt  
import cartopy 
import cartopy.crs as ccrs
from glob import glob
from scipy import stats
from scipy.stats import ttest_1samp, wilcoxon, ttest_ind, mannwhitneyu
import itertools
from itertools import combinations


%matplotlib inline
# Some defaults
plt.rcParams['figure.figsize'] = (14, 5)  # Default plot size
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are not important
import warnings
warnings.filterwarnings('ignore')

In [2]:
# settings for plots for latex
fig_width_pt = 345.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]

In [3]:
ds_dry = xr.open_mfdataset('/home/miri/Dokumente/Master/MA/ERA-Interim and Spells/DrySpell.nc')
ds_dry_sig = xr.open_mfdataset('/home/miri/Dokumente/Master/MA/ERA-Interim and Spells/Dry_Spell_std.nc')
ds_wet = xr.open_mfdataset('/home/miri/Dokumente/Master/MA/ERA-Interim and Spells/WetSpell.nc')
ds_wet_sig = xr.open_mfdataset('/home/miri/Dokumente/Master/MA/ERA-Interim and Spells/Wet_Spell_std.nc')

ds = xr.open_dataset('/home/miri/Dokumente/Master/MA/Data/data_DJFM_mean.nc')
dstd = xr.open_dataset('/home/miri/Dokumente/Master/MA/Data/data_DJFM_std.nc')
ds_z = xr.open_dataset('/home/miri/Dokumente/Master/MA/Data/data_mmodm_INVARIANT_new.nc')

plot_loc = ('/home/miri/Dokumente/Master/MA/Doc/Plots/Anomaly/')

In [4]:
shdf = salem.read_shapefile(salem.get_demo_file('world_borders.shp'))
shdf = shdf.loc[shdf['CNTRY_NAME'].isin(['Peru','Brazil','Bolivia','Ecuador'])]  # GeoPandas' GeoDataFrame
ds_sub = ds.salem.subset(shape=shdf, margin=2)  # add 2 grid points
dstd_sub = dstd.salem.subset(shape=shdf, margin=2) 
# mask roi = region of interest
#ds_sub = ds_sub.salem.roi(shape=shdf) #wird nur ROI betrachtet --> Maskierung der Region von Interesse


In [5]:
# global parameters
# length of dataset
N_dry = 947 # Dry
N_wet = 997 # Wet
N_all = 4487

# levels 
levels = [200, 500, 850]
#levels2 = [700, 850]

# selected latitude and longitude for the various seasons
lon= -77.25
lat= -9.75


In [6]:
# slice east, west, north, south boarders
sliceE = lon+40*0.75
sliceW = lon-25*0.75
sliceN = lat +25*0.75
sliceS = lat -18*0.75

# slice east, west, north, south boarders
sliceEZ = lon+20*0.75
sliceWZ = lon-5*0.75
sliceNZ = lat +10*0.75
sliceSZ = lat -10*0.75

sliceE_CS = lon+20*0.75
sliceW_CS = lon-5*0.75
sliceN_CS = lat +10*0.75
sliceS_CS = lat -10*0.75

In [7]:
height = levels
height = list(map(int, height))
level = list(map(int, ds.level))

from collections import OrderedDict # ordered dictionaries for loops



outdatam = OrderedDict()
for h in height:
    d = dict()
    d['u_h'] = ds.u.sel(level=h)
    d['v_h'] = ds.v.sel(level=h)
    d['w_h'] = ds.w.sel(level=h)
    d['z_h'] = ds.z.sel(level=h)
    d['t_h'] = ds.t.sel(level=h)
    d['r_h'] = ds.r.sel(level=h)
    d['q_h'] = ds.q.sel(level=h)
    outdatam[h] = d
    
# ordered dictionary for std ds
outdatas = OrderedDict()
for h in height:
    d = dict()
    d['u_h'] = dstd.u.sel(level=h)
    d['v_h'] = dstd.v.sel(level=h)
    d['w_h'] = dstd.w.sel(level=h)
    d['z_h'] = dstd.z.sel(level=h)
    d['t_h'] = dstd.t.sel(level=h)
    d['r_h'] = dstd.r.sel(level=h)
    d['q_h'] = dstd.q.sel(level=h)
    outdatas[h] = d


# ordered dictionary for dry ds
drydata = OrderedDict()
for h in height:
    d = dict()
    d['u_h'] = ds_dry.u.sel(level=h)
    d['v_h'] = ds_dry.v.sel(level=h)
    d['w_h'] = ds_dry.w.sel(level=h)
    d['z_h'] = ds_dry.z.sel(level=h)
    d['t_h'] = ds_dry.t.sel(level=h)
    d['r_h'] = ds_dry.r.sel(level=h)
    d['q_h'] = ds_dry.q.sel(level=h)
    drydata[h] = d
    
    
# ordered dictionary for wet ds
wetdata = OrderedDict()
for h in height:
    d = dict()
    d['u_h'] = ds_wet.u.sel(level=h)
    d['v_h'] = ds_wet.v.sel(level=h)
    d['w_h'] = ds_wet.w.sel(level=h)
    d['z_h'] = ds_wet.z.sel(level=h)
    d['t_h'] = ds_wet.t.sel(level=h)
    d['r_h'] = ds_wet.r.sel(level=h)
    d['q_h'] = ds_wet.q.sel(level=h)
    wetdata[h] = d

# ordered dictionary for dry ds
drydata_sig = OrderedDict()
for h in height:
    d = dict()
    d['u_h'] = ds_dry_sig.u.sel(level=h)
    d['v_h'] = ds_dry_sig.v.sel(level=h)
    d['w_h'] = ds_dry_sig.w.sel(level=h)
    d['z_h'] = ds_dry_sig.z.sel(level=h)
    d['t_h'] = ds_dry_sig.t.sel(level=h)
    d['r_h'] = ds_dry_sig.r.sel(level=h)
    d['q_h'] = ds_dry_sig.q.sel(level=h)
    drydata_sig[h] = d
    
    
# ordered dictionary for wet ds
wetdata_sig = OrderedDict()
for h in height:
    d = dict()
    d['u_h'] = ds_wet_sig.u.sel(level=h)
    d['v_h'] = ds_wet_sig.v.sel(level=h)
    d['w_h'] = ds_wet_sig.w.sel(level=h)
    d['z_h'] = ds_wet_sig.z.sel(level=h)
    d['t_h'] = ds_wet_sig.t.sel(level=h)
    d['r_h'] = ds_wet_sig.r.sel(level=h)
    d['q_h'] = ds_wet_sig.q.sel(level=h)
    wetdata_sig[h] = d

In [42]:
def welch_t_test(mu1, s1, N1, mu2, s2, N2):
    """http://en.wikipedia.org/wiki/Welch%27s_t_test"""
    
    mu1 = np.asarray(mu1)
    mu2 = np.asarray(mu2)
    s1 = np.asarray(s1)
    s2 = np.asarray(s2)
    
    if not np.allclose(mu1.shape, mu2.shape):
        raise ValueError('mu1 and mu2 should have the same shape')
    
    if not np.allclose(s1.shape, s2.shape):
        raise ValueError('s2 and s2 should have the same shape')
    
    if not mu1.shape:
        # Construct arrays to make calculations more succint.
        N_i = np.array([N1, N2])
        dof_i = N_i - 1
        v_i = np.array([s1, s2]) ** 2
        # Calculate t-stat, degrees of freedom, use scipy to find p-value.
        t = (mu1 - mu2) / np.sqrt(np.sum(v_i / N_i))
        dof = (np.sum(v_i / N_i) ** 2) / np.sum((v_i ** 2) / ((N_i ** 2) * dof_i))
        p = stats.distributions.t.sf(np.abs(t), dof) * 2
        return t, p
    else:
        ps = []
        ts = []
        for _mu1, _s1, _mu2, _s2 in zip(mu1.flatten(), s1.flatten(), mu2.flatten(), s2.flatten()):
            t, p = welch_t_test(_mu1, _s1,  N1, _mu2,_s2, N2)  
            ps.append(p)
            ts.append(t)
        return np.asarray(ts).reshape(mu1.shape), np.asarray(ps).reshape(mu1.shape)

In [43]:
def fdr_threshold(pvalues, alpha=0.05):
    """Computes the FDR threshod after Wilks (2016)."""
    p = np.sort(np.asarray(pvalues).flatten())
    n = len(p)
    return np.max(np.where(p <= (np.arange(1, n+1) / n * alpha), p, 0))

In [25]:
lons, lats = ds.salem.grid.ll_coordinates  # np.meshgrid

In [44]:
# welsh test u wind component
# all levels
dry_u = ds_dry.u
dry_usig = ds_dry_sig.u
wet_u = ds_wet.u
wet_usig = ds_wet_sig.u
clim_u = ds.u
clim_usig = dstd.u

out_u_dry = [] 
for mu1, s1, mu2, s2 in zip(dry_u, dry_usig, clim_u, clim_usig):
    t_udry, p_udry = welch_t_test(mu1, s1, N_dry, mu2, s2, N_all)

out_u_wet = [] 
for mu1, s1, mu2, s2 in zip(wet_u, wet_usig, clim_u, clim_usig):
    t_uwet, p_uwet = welch_t_test(mu1, s1, N_wet, mu2, s2, N_all)
    

In [49]:
dry_u.shape

(12, 68, 162)

In [46]:
p_udry.shape

(68, 162)

In [67]:



dry_u = ds_dry.u
dry_usig = ds_dry_sig.u
wet_u = ds_wet.u
wet_usig = ds_wet_sig.u
clim_u = ds.u
clim_usig = dstd.u

out_u_dry = [] 
for mu1, s1, mu2, s2 in zip(dry_u, dry_usig, clim_u, clim_usig):
    t_udry, p_udry = welch_t_test(mu1, s1, N_dry, mu2, s2, N_all)

out_u_wet = [] 
for mu1, s1, mu2, s2 in zip(wet_u, wet_usig, clim_u, clim_usig):
    t_uwet, p_uwet = welch_t_test(mu1, s1, N_wet, mu2, s2, N_all)

    

In [71]:
dry_u.shape

(12, 68, 162)

In [69]:
mu1

<xarray.DataArray 'u' (latitude: 68, longitude: 162)>
array([[-6.65993881, -6.52154255, -6.38559151, ..., -3.52422786,
        -3.2020781 , -3.06937599],
       [-6.95741224, -6.835711  , -6.69778872, ..., -3.79625797,
        -3.45649719, -3.29211068],
       [-7.20594645, -7.09165049, -6.9596324 , ..., -3.77786493,
        -3.452106  , -3.36447883],
       ..., 
       [-2.66413713, -2.65651417, -2.66010809, ..., -2.18486476,
        -2.23656917, -2.31308746],
       [-2.41791916, -2.40983224, -2.40652227, ..., -1.6215589 ,
        -1.67721593, -1.75770438],
       [-2.17435551, -2.16784883, -2.15592766, ..., -1.0886718 ,
        -1.15179086, -1.20399833]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -130.5 -129.75 -129.0 -128.25 -127.5 ...
  * latitude   (latitude) float32 20.25 19.5 18.75 18.0 17.25 16.5 15.75 ...
    level      int32 1000

In [72]:
p_udry.shape

(68, 162)

In [30]:
# welsh test v wind component
# all levels
dry_v = ds_dry.v
dry_vsig = ds_dry_sig.v
wet_v = ds_wet.v
wet_vsig = ds_wet_sig.v
clim_v = ds.v
clim_vsig = dstd.v

out_v_dry = [] 
for mu1, s1, mu2, s2 in zip(dry_v.values, dry_vsig.values, clim_v.values, clim_vsig.values):
    t_vdry, p_vdry = welch_t_test(mu1, s1, N_dry, mu2, s2, N_all)
    

out_v_wet = [] 
for mu1, s1, mu2, s2 in zip(wet_v, wet_vsig, clim_v, clim_vsig):
    t_vwet, p_vwet = welch_t_test(mu1, s1, N_wet, mu2, s2, N_all)
    

In [31]:
mu1.shape

(68, 162)

In [32]:
dry_v.shape

(12, 68, 162)

In [34]:
p_vdry.shape

(68, 162)